In [1]:
from enum import Enum

class RerankType(Enum):
    RRF = 0             # 상호순위조합(Reciprocal Ranking Fusion, RRF)
    CrossEncoder = 1
    RerankerModel = 2

class EmbeddingType(Enum):
    OPEN_AI = 0
    SOLAR = 1

In [2]:
# True : 문서/청크 임베딩을 피클 파일에서 읽음
# False : 임베딩을 새로함
is_use_content_embeddings_from_pickle = True
is_use_chunk_embeddings_from_pickle = True

embeddingType = EmbeddingType.OPEN_AI

# 문서 임베딩의 피클 파일명
if embeddingType == EmbeddingType.OPEN_AI:
    content_embeddings_pickle_filename = 'content_embeddings_text-embedding-3-large_dim3072.pkl'
    chunk_embeddings_pickle_filename = 'chunk_300_embeddings_text-embedding-3-large_dim3072.pkl'
    
#elif embeddingType == EmbeddingType.SOLAR:  embeddings_pickle_filename = 'embeddings_solar_embedding-passage_dim4096.pkl'
elif embeddingType == EmbeddingType.SOLAR:  embeddings_pickle_filename = 'embeddings_solar-embedding-1-large-passage_dim4096.pkl'
else:
    assert False, "Wrong embedding type!"

# 문서 리랭크 방법
rerankType = RerankType.RerankerModel

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [4]:
 # 색인 대상 문서 및 평가 데이터 다운로드
 #!wget https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000291/data/data.tar.gz
 #!tar -xzvf data.tar.gz

## 환경 설정
검색엔진을 위한 Elasitcsearch, 임베딩 생성을 위한 sentence transformers, LLM 사용을 위한 openai client 설치가 필요합니다.

In [5]:
# Elasticsearch 8.8.0 다운로드 및 압축 해제
#!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.8.0-linux-x86_64.tar.gz
#!tar -xzf elasticsearch-8.8.0-linux-x86_64.tar.gz

# daemon user로 구동을 하기 위해 소유자 변경
#!chown -R daemon:daemon elasticsearch-8.8.0/

# 코랩 노트북 환경에서 서버 구동을 위한 리소스 제한/격리를 위해 아래 명령 수행
#!umount /sys/fs/cgroup
#!apt install cgroup-tools

# Nori 형태소 분석기 설치
#!./elasticsearch-8.8.0/bin/elasticsearch-plugin install analysis-nori


In [6]:
# Elasticsearch python 패키지 설치
#!pip install elasticsearch==8.15.1

# OpenAI Python 패키지 설치
#!pip install openai==1.7.2

# 임베딩 생성을 위한 벡터 인코더 설치
#!pip install sentence-transformers==2.2.2

## 검색엔진 준비 - Elasticsearch




In [7]:
import os
import json
from elasticsearch import Elasticsearch, helpers
import time

In [8]:

# 엘라스틱서치의 데몬 인스턴스 만들기
# from subprocess import Popen, PIPE, STDOUT

# es_server = Popen(['../../elasticsearch-8.8.0/bin/elasticsearch'],
#                   stdout=PIPE, stderr=STDOUT,
#                   preexec_fn=lambda: os.setuid(1)  # as daemon
#                  )

# 인스턴스를 로드하는 데 약간의 시간이 걸림
#time.sleep(60)

In [ ]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

In [10]:
# 데몬 구동후 password 설정 단계 필요
# 명령 실행 후 "Please confirm that you would like to continue"에서 y 입력 필요
#!/content/elasticsearch-8.8.0/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"

In [ ]:
es_username = 'elastic'

# 위 명령 실행 결과의 마지막 부분인 PASSWORD elastic 값으로 교체 필요
es_password = os.getenv('ES_PASSWORD')

# Elasticsearch client 생성
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="../../elasticsearch-8.15.2/config/certs/http_ca.crt")

# Elasticsearch client 정보 확인
print(es.info())

In [12]:
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["OPENAI_API_KEY"] = "Your API Key"

clientForOpenAi = OpenAI()
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"

clientForSolar = OpenAI(
    api_key=os.getenv('UPSTAGE_API_KEY'),
    base_url="https://api.upstage.ai/v1/solar"
)

In [ ]:
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
#model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
#model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# SetntenceTransformer를 이용하여 임베딩 생성
# def get_embedding(sentences):
#     return model.encode(sentences)


def get_query_embedding(text):
    if embeddingType == EmbeddingType.OPEN_AI:
        return clientForOpenAi.embeddings.create(input=[text], model="text-embedding-3-large").data[0].embedding
    elif embeddingType == EmbeddingType.SOLAR:
        response = clientForSolar.embeddings.create(
            input=text,
            #model="embedding-query"
            model="solar-embedding-1-large-query"
        )
        return response.data[0].embedding
    else:
        assert False, "Wrong embedding type!!"


def get_passage_embedding(text):
    if embeddingType == EmbeddingType.OPEN_AI:
        return clientForOpenAi.embeddings.create(input=[text], model="text-embedding-3-large").data[0].embedding
    elif embeddingType == EmbeddingType.SOLAR:
        response = clientForSolar.embeddings.create(
            input=text,
            #model="embedding-passage"
            model="solar-embedding-1-large-passage"
        )
        return response.data[0].embedding
    else:
        assert False, "Wrong embedding type!!"


def get_passages_embeddings(texts):
    embeddings = [get_passage_embedding(text) for text in texts]
    return embeddings


# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_passages_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc["content"] for doc in batch]
        
        embeddings = get_passages_embeddings(contents)
        
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings


# 새로운 index 생성
def create_es_index(index, settings, mappings):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, settings=settings, mappings=mappings)


# 지정된 인덱스 삭제
def delete_es_index(index):
    es.indices.delete(index=index)


# Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    return helpers.bulk(es, actions)


# 역색인을 이용한 검색
def sparse_retrieve(query_str, size):
    query = {
        "match": {
            "content": {
                "query": query_str
            }
        }
    }
    return es.search(index="test", query=query, size=size, sort="_score")       ######## index 를 "test" 로 박아놨네??


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_query_embedding(query_str)

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": "embedding",
        "query_vector": query_embedding,
        "k": size,
        "num_candidates": 100
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", knn=knn)     ######## index 를 "test" 로 박아놨네??

In [14]:
# 문서의 content 필드에 대한 임베딩 생성
with open("../../data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]

with open("../../data/chunked_documents_300.jsonl") as f:
    chunks = [json.loads(line) for line in f]    

In [ ]:
# docid 로 content 를 가져올 수 있게 딕셔너리 생성
docs_dic = {}

for doc in docs:
    key = doc['docid']
    val = doc['content']
    
    docs_dic[key] = val

docs_dic["42508ee0-c543-4338-878e-d98c6babee66"]

In [ ]:
chunks[0]

In [17]:
# ColBERT 에서 사용
docs_only_content = [doc['content'] for doc in docs]

In [18]:
import pickle

# 문서
if not is_use_content_embeddings_from_pickle:
    content_embeddings = get_passages_embeddings_in_batches(docs)
    
    # 나중을 위해 문서 임베딩을 피클 파일로 저장.
    with open(content_embeddings_pickle_filename, 'wb') as file:
        pickle.dump(content_embeddings, file)
else:
    # 피클 파일로부터 embeddings 리스트 복원
    with open(content_embeddings_pickle_filename, 'rb') as file:
        content_embeddings = pickle.load(file)

In [ ]:
print(f'content embedding dimension : {len(content_embeddings[0])}')
print(f'content embeddings type : {type(content_embeddings)}')
print(f'content embeddings length : {len(content_embeddings)}')
print(f'content embeddings[0] : {content_embeddings[0]}')

In [20]:
# 청크
if not is_use_chunk_embeddings_from_pickle:
    chunk_embeddings = get_passages_embeddings_in_batches(chunks)
    
    # 나중을 위해 문서 임베딩을 피클 파일로 저장.
    with open(chunk_embeddings_pickle_filename, 'wb') as file:
        pickle.dump(chunk_embeddings, file)
else:
    # 피클 파일로부터 embeddings 리스트 복원
    with open(chunk_embeddings_pickle_filename, 'rb') as file:
        chunk_embeddings = pickle.load(file)

In [ ]:
print(f'chunk embedding dimension : {len(chunk_embeddings[0])}')
print(f'chunk embeddings type : {type(chunk_embeddings)}')
print(f'chunk embeddings length : {len(chunk_embeddings)}')
print(f'chunk embeddings[0] : {chunk_embeddings[0]}')

In [22]:
# 색인을 위한 setting 설정
settings = {
    "index": {
            "similarity": {
                "lm_jelinek_mercer": {
                    "type": "LMJelinekMercer",
                    "lambda": 0.7
                }
            }
    },
    "analysis": {
        "analyzer": {
            "nori_index_analyzer": {
                "type": "custom",
                "tokenizer": "nori_tokenizer",# Elasticsearch에서 제공하는 한국어 형태소 분석기로, 텍스트를 의미 있는 단어 단위로 분리합니다.
                "decompound_mode": "mixed", # 복합어를 어떻게 처리할지 결정합니다. 'mixed' 설정은 원래 형태와 분리된 형태 모두를 토큰으로 생성합니다.
                "user_dictionary":"/data/ephemeral/home/elasticsearch-8.15.2/config/synonyms/user_dictionary.txt",
                "filter": ["nori_posfilter"]
            },
            "nori_search_analyzer": {
                "type": "custom",
                "tokenizer": "nori_tokenizer",
                "filter": ["nori_posfilter",
                           "my_synonym_filter1",
                           #"my_synonym_filter2", # 에러나서 뻄
                           "my_synonym_filter3",
                           "my_synonym_filter4",
                           "my_synonym_filter5",
                           "my_synonym_filter6",
                           "my_synonym_filter7"]  # 검색 시점에서 동의어 필터 사용
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                "stoptags": ["E", "J", "SC", "SE", "SF", "VCN" ]    # 여기서 지정한 것들을 색인하지 않음!
            },
            "my_synonym_filter1": {
                "type": "synonym",
                "synonyms_path": "/data/ephemeral/home/elasticsearch-8.15.2/config/synonyms/synonyms1.txt",
            },
            # "my_synonym_filter2": {
            #     "type": "synonym",
            #     "synonyms_path": "/data/ephemeral/home/elasticsearch-8.15.2/config/synonyms/synonyms2.txt",
            # },
            "my_synonym_filter3": {
                "type": "synonym",
                "synonyms_path": "/data/ephemeral/home/elasticsearch-8.15.2/config/synonyms/synonyms3.txt",
            },
            "my_synonym_filter4": {
                "type": "synonym",
                "synonyms_path": "/data/ephemeral/home/elasticsearch-8.15.2/config/synonyms/synonyms4.txt",
            },
            "my_synonym_filter5": {
                "type": "synonym",
                "synonyms_path": "/data/ephemeral/home/elasticsearch-8.15.2/config/synonyms/synonyms5.txt",
            },
             "my_synonym_filter6": {
                "type": "synonym",
                "synonyms_path": "/data/ephemeral/home/elasticsearch-8.15.2/config/synonyms/synonyms6.txt",
            },
              "my_synonym_filter7": {
                "type": "synonym",
                "synonyms_path": "/data/ephemeral/home/elasticsearch-8.15.2/config/synonyms/synonyms7.txt",
            }
        }
    }
}

# 색인을 위한 mapping 설정 (역색인 필드, 임베딩 필드 모두 설정)
mappings = {
    "properties": {
        "content": {
            "type": "text",
            "analyzer": "nori_index_analyzer",       # 색인 시점 분석기
            "search_analyzer": "nori_search_analyzer" # 검색 시점 분석기
        },
        "embedding": {
            "type": "dense_vector",
            "dims": len(chunk_embeddings[0]),
            "index": True,
            "similarity": "l2_norm"
        }
    }
}

In [23]:
# settings, mappings 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", settings, mappings)

In [ ]:
# 생성한 임베딩을 색인할 필드로 추가
index_chunks = []
for chunk, embedding in zip(chunks, chunk_embeddings):
    chunk["embedding"] = embedding
    index_chunks.append(chunk)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_chunks)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)

In [25]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
test_query = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"

In [ ]:
# 역색인을 사용하는 검색 예제
search_result_retrieve = sparse_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

In [ ]:
# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

## ColBERT

In [28]:
# 필요한 라이브러리 import
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

In [ ]:
# ColBERTv2.ipynb 에서 학습한 모델의 위치를 checkpoint에 넣어줌
checkpoint = 'experiments/training/none/2024-10/22/10.44.21/checkpoints/colbert'
#checkpoint = 'Kerneld/sentence-transformer-klue-temp'
experiment = 'after_trained'
index_name = 'science_common_sense'
nbits = 2   # encode each dimension with 2 bits

with Run().context(RunConfig(nranks=1, experiment=experiment)):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.
    indexer = Indexer(checkpoint = checkpoint, config = config)
    indexer.index(name = index_name, collection = docs_only_content, overwrite=True)

In [30]:
def search_docs_from_ColBERT(query, experiment, index_name, k):
    with Run().context(RunConfig(experiment = experiment)):
        searcher = Searcher(index = index_name, collection = docs_only_content)

    # Find the top-3 passages for this query
    results = searcher.search(query, k)
    
    ret = []

    # Print out the top-k retrieved passages
    for passage_id, passage_rank, passage_score in zip(*results):
        print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")
        
        dic = {
            'docid': docs[passage_id]['docid'],
            'content': docs[passage_id]['content'],
            'embedding': content_embeddings[passage_id],
        }
        ret.append(dic)
    
    return ret

In [ ]:
ret = search_docs_from_ColBERT(test_query, experiment, index_name, 3)
print(ret)

## RAG 구현

준비된 검색엔진과 LLM을 활용하셔 대화형 RAG 구현

In [32]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- search api를 호출할 때, 사용자의 대화가 멀티턴이 아니라면, 사용자의 대화 내용을 그대로 standalone_query 에 대입한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "Final query suitable for use in search from the user messages history."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]


In [33]:
from sentence_transformers import CrossEncoder
#cross_encoder_model = CrossEncoder("bongsoo/albert-small-kor-cross-encoder-v1")
#cross_encoder_model = CrossEncoder("Kerneld/klue-roberta-small-cross-encoder")
cross_encoder_model = CrossEncoder("Kerneld/klue-roberta-small-cross-encoder-temp")

In [34]:
def convert_elasticsearch_result_to_common_result(search_results):
    ret_list = []
    
    for rst in search_results['hits']['hits']:
        dic = {}
        
        dic['docid'] = rst['_source']['docid']
        dic['content'] = rst['_source']['content']
        dic['embedding'] = rst['_source']['embedding']
        
        ret_list.append(dic)
        
    return ret_list

In [35]:
def convert_colbert_result_to_common_result(search_results):
    return search_results

In [36]:
from collections import defaultdict
from typing import List

def reciprocal_rank_fusion(common_results_list:List[List[dict]], k=5):
    rrf = {}
    
    for common_results in common_results_list:
        for i, common_result in enumerate(common_results, 1):
            key = common_result['docid']
            
            if key not in rrf:
                dic = defaultdict(float)
                rrf[key] = dic
            else:
                dic = rrf[key]
            
            dic['score'] += 1.0 / (k + i)
            dic['content'] = common_result['content']
    
    return sorted(rrf.items(), key=lambda x: x[1]['score'], reverse=True)

In [37]:
def append_without_duplicate(*src_list_list):
    ret_list = []
    
    for src_list in src_list_list:
        for src_rst in src_list:
            # TODO 검색 코드로 바꿔도 될듯?
            is_found = False
            
            for dst_rst in ret_list:
                if src_rst['docid'] == dst_rst['docid']:
                    is_found = True
                    break
            
            if is_found: continue
            
            ret_list.append(src_rst)              
    
    return ret_list

In [38]:
def convert_common_results_to_rank_infos(common_results_no_dup):
    ret = []
    
    for rst in common_results_no_dup:
        dic = {
            'score': 0.0, 
            'content': rst['content']
        }
        
        ret.append( (rst['docid'], dic) )
    
    return ret

In [39]:
import numpy as np

def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()

In [40]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

reranker_tokenizer = AutoTokenizer.from_pretrained("Dongjin-kr/ko-reranker")
reranker_model = AutoModelForSequenceClassification.from_pretrained("Dongjin-kr/ko-reranker")
reranker_model.eval()

def get_rank_infos(standalone_query):
    results_sparse = sparse_retrieve(standalone_query, 20)
    results_dense = dense_retrieve(standalone_query, 20)
    result_colbert = search_docs_from_ColBERT(standalone_query, experiment, index_name, 20)
    
    common_results_1 = convert_elasticsearch_result_to_common_result(results_sparse)
    common_results_2 = convert_elasticsearch_result_to_common_result(results_dense)
    common_results_3 = convert_colbert_result_to_common_result(result_colbert)
    
    if rerankType == RerankType.RRF:
        # 상호순위조합(RRF 로 정렬)
        common_results_list = [common_results_1, common_results_2, common_results_3]
        rank_infos = reciprocal_rank_fusion(common_results_list)
    
    elif rerankType == RerankType.CrossEncoder:
        # CrossEncoder 로 정렬
        
        # 중복된 docid 제거
        common_results_no_dup = append_without_duplicate(common_results_1, common_results_2, common_results_3)
        
        input_pairs = [
            [standalone_query, rst['content']]
            for rst in common_results_no_dup
        ]
        
        relevance_scores = cross_encoder_model.predict(input_pairs)
        new_order = np.argsort(relevance_scores)[::-1]
        
        common_results_no_dup = np.array(common_results_no_dup)
        common_results_no_dup = common_results_no_dup[new_order]
        
        rank_infos = convert_common_results_to_rank_infos(common_results_no_dup)
    
    elif rerankType == RerankType.RerankerModel:
        # 중복된 docid 제거
        common_results_no_dup = append_without_duplicate(common_results_1, common_results_2, common_results_3)
        
        input_pairs = [
            [standalone_query, rst['content']]
            for rst in common_results_no_dup
        ]
        
        with torch.no_grad():
            inputs = reranker_tokenizer(input_pairs, 
                                        padding=True, 
                                        truncation=True, 
                                        return_tensors='pt', 
                                        max_length=512).to('cuda')
            reranker_model.to('cuda')
            scores = reranker_model(**inputs, return_dict=True).logits.view(-1).float()
            
            # scores 의 모든 원소의 합은 거의 1에 근접함.
            scores = exp_normalize(scores.cpu().numpy())
        
        new_order = np.argsort(scores)[::-1]
        
        common_results_no_dup = np.array(common_results_no_dup)
        common_results_no_dup = common_results_no_dup[new_order]
        
        rank_infos = convert_common_results_to_rank_infos(common_results_no_dup)
        
    else:
        assert(False, "wrong rerank type!")
    
    return rank_infos

In [41]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = clientForOpenAi.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("standalone_query")
        response["standalone_query"] = standalone_query
        
        rank_infos = get_rank_infos(standalone_query)
        
        retrieved_context = []
        for i, rank_info in enumerate(rank_infos):
            if i >= 20:
                break   # 최대 20개까지만..
            
            docid = rank_info[0]
            content = rank_info[1]['content']
            score = rank_info[1]['score']
            
            retrieved_context.append(content)
            
            response["topk"].append(docid)
            response["references"].append({"score": score, "content": content})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = clientForOpenAi.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1,
                    timeout=30
                )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response

In [42]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question_with_generated_standalone_query(standalone_query):
    # 함수 출력 초기화
    response = {"standalone_query": standalone_query, "topk": [], "references": [], "answer": ""}

    if standalone_query != '':
        rank_infos = get_rank_infos(standalone_query)
        
        for i, rank_info in enumerate(rank_infos):
            if i >= 20:
                break   # 최대 20개까지만..
            
            docid = rank_info[0]
            content = rank_info[1]['content']
            score = rank_info[1]['score']
            
            response["topk"].append(docid)
            response["references"].append({"score": score, "content": content})

    return response

In [43]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            # if idx > 0:
            #     break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

In [44]:
# 생성된 standalone_query 를 사용해서 결과 추출후 파일에 저장
def eval_rag_with_generated_standalone_query(standalone_query_filename, output_filename):
    with open(standalone_query_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            # if idx > 0:
            #     break
            j = json.loads(line)
            print(f'Test {idx}\Standalone query: {j["standalone_query"]}')
            response = answer_question_with_generated_standalone_query(j["standalone_query"])

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": j["standalone_query"], "topk": response["topk"], "answer": "", "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

In [ ]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
submission_filename = "submission_033.csv"

#eval_rag("../../data/eval.jsonl", submission_filename)
#eval_rag("../../data/valid_230_exclude_smalltalk.jsonl", submission_filename)
#eval_rag_with_generated_standalone_query('standalone_query_from_submission.jsonl', submission_filename)
eval_rag_with_generated_standalone_query('standalone_query_from_submission_CJM.jsonl', submission_filename)